In [10]:
from ANEMO import ANEMO
from ANEMO import read_edf

import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Récupération des données

In [11]:
datafile = 'data/enregistrement_AM_2017-10-23_100057.asc'

try:
    data = read_edf(datafile, 'TRIALID')
except:
    from urllib.request import urlopen
    url = 'https://github.com/chloepasturel/AnticipatorySPEM/raw/master/data/enregistrement_AM_2017-10-23_100057.asc'
    
    with open(datafile, 'wb') as f:
        f.write(urlopen(url).read())
    urlopen(url).close()
    f.close()
    data = read_edf(datafile, 'TRIALID')
    


expfile = 'data/enregistrement_AM_2017-10-23_100057.pkl'

try :
    with open(expfile, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')
except :
    from urllib.request import urlopen
    url = 'https://github.com/chloepasturel/AnticipatorySPEM/raw/master/data/enregistrement_AM_2017-10-23_100057.pkl'
    
    with open(expfile, 'wb') as f:
        f.write(urlopen(url).read())
    urlopen(url).close()
    f.close()
    with open(expfile, 'rb') as fichier :
        exp = pickle.load(fichier, encoding='latin1')

In [12]:
A = ANEMO(exp)

# Fonction

In [13]:
def fct_velocity (x, dir_target, start_anti, v_anti, latence, tau, maxi, do_whitening) :
    v_anti = v_anti/1000 # pour passer de sec à ms
    time = x # np.arange(len(x))
    vitesse = []
    y = 0

    for t in range(len(time)):

        if start_anti >= latence :
            if time[t] < latence :
                vitesse.append(0)
            else :
                vitesse.append(dir_target*maxi*(1-np.exp(-1/tau*(time[t]-latence))))
        else :
            if time[t] < start_anti :
                vitesse.append(0)
            else :
                if time[t] < latence :
                    vitesse.append((time[t]-start_anti)*v_anti)
                    y = (time[t]-start_anti)*v_anti
                else :
                    vitesse.append(dir_target*maxi*(1-np.exp(-1/tau*(time[t]-latence)))+y)

    if do_whitening is True:
        vitesse = whitening(vitesse)

    return vitesse

In [15]:
data_trial = data[0]
arg = A.arg(data_trial)

In [16]:
max_latence = []
for s in range(len(arg.saccades)) :
    if (arg.saccades[s][0]-arg.t_0) >= (arg.TargetOn-arg.t_0+100) :
        max_latence.append((arg.saccades[s][0]-arg.t_0))
if max_latence==[] :
    max_latence.append(len(arg.trackertime))
max_latence = max_latence[0]

param_fit=[{'name':'tau', 'value':15., 'min':13., 'max':80., 'vary':'vary'},
           {'name':'maxi', 'value':15, 'min':1., 'max':40., 'vary':True},
           {'name':'dir_target', 'value':arg.dir_target, 'min':None, 'max':None, 'vary':False},
           {'name':'v_anti', 'value':0., 'min':-40., 'max':40., 'vary':'vary'},
           {'name':'latence', 'value':arg.TargetOn-arg.t_0+100, 'min':arg.TargetOn-arg.t_0+75, 'max':max_latence, 'vary':True},
           {'name':'start_anti', 'value':arg.TargetOn-arg.t_0-100, 'min':arg.StimulusOf-arg.t_0-200, 'max':arg.TargetOn-arg.t_0+75, 'vary':'vary'}]


inde_vars={'x':np.arange(len(arg.trackertime))}

  

In [17]:
Plot = ANEMO.Plot(exp)

In [18]:
trial=11
block=1

## Plot velocity

In [20]:
fig, axs = Plot.plot_equation(fct_velocity(inde_vars, param_fit))

TypeError: fct_velocity() missing 6 required positional arguments: 'start_anti', 'v_anti', 'latence', 'tau', 'maxi', and 'do_whitening'

# Enregistrement figure

In [18]:
Fit = ANEMO.Fit(exp)

In [ ]:
param = Fit.Fit_full(data[:30], equation='fct_velocity', plot=True,
                     N_blocks=3, N_trials=10, file_fig='result/Fit_velocity')

# enregistre les paramêtres des fits
with open('result/param_Fit_AM.pkl', 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(param)

block, trial =  0 0
block, trial =  0 1
block, trial =  0 2
block, trial =  0 3
block, trial =  0 4
block, trial =  0 5
block, trial =  0 6
block, trial =  0 7
block, trial =  0 8
block, trial =  0 9
block, trial =  1 0
block, trial =  1 1
block, trial =  1 2
block, trial =  1 3
block, trial =  1 4
block, trial =  1 5
block, trial =  1 6
